Technical OA I Passed a while back

Problem #1

In [1]:
#using pandas for data manipulation due to convenience and familiarity
import pandas as pd

#opening datasets into dataframes
upcoming_repairs = pd.read_csv('upcoming_repairs.csv')
technicians = pd.read_csv('technicians.csv')
repair_types = pd.read_csv('repair_types.csv')

#merges the repair value and time from repair_types dataframe with the upcoming repairs dataframe
merged_repairs = upcoming_repairs.merge(repair_types[['repair_name', 'repair_value', 'time_in_minutes']], on='repair_name', how='left')

#establishes adjusted value as severity of repair * base repair value of type of repair
merged_repairs['adjusted_value'] = merged_repairs['severity'] * merged_repairs['repair_value']
#identifies the adjusted value per minute of labor for each repair
merged_repairs['relative_value'] = (merged_repairs['severity'] * merged_repairs['repair_value']) / merged_repairs['time_in_minutes']

#function to convert shift times to minutes after midnight
#this would fail if there were overnight shifts or a 12 AM start/end time
def to_minutes(time):
    #checks if shift starts or ends at an exact hour so it can reformat it to all fit ab:cd(AM/PM) format
    if len(time) <= 4:
        time = time[:-2] + ':00' + time[-2:]
    #establishes the AM or PM part of the time as the period
    period = time[-2:]
    #takes the numerical part of the time as the time
    time = time[:-2]
    #takes the hours and minutes as the numbers on either side of the :
    hours,minutes = map(int, time.split(':'))
    #adjusts the hours since midnight for pm times
    if period =='PM' and hours != 12:
        hours += 12
    return hours * 60 + minutes

#uses to_minutes function to establish the start and end shift times with minutes after midnight
technicians['start_minutes'] = technicians['start_time'].apply(to_minutes)
technicians['end_minutes'] = technicians['end_time'].apply(to_minutes)
#calculates shift length in minutes
technicians['shift_length'] = technicians['end_minutes'] - technicians['start_minutes']
#calculates minutes of labor that each employee works over the next 2 weeks
technicians['labor_minutes'] = technicians['shift_length'] * (technicians['number_of_days'] * 2)

#our assumptions let us allocate the employees so efficiently that we can abstract them as a pool of labor to draw from
#calculates the labor constraint as the total number of labor minutes available for repairs
total_labor_minutes = technicians['labor_minutes'].sum()

#sorts repairs by the highets adjusted value per minute
merged_repairs = merged_repairs.sort_values(by='relative_value', ascending=False)
#initializes a sum of the adjusted value generated by the repairs so far
sum_repair_value=0
#initializes a sum of the labor used to make the repairs so far
sum_labor_used=0
#loops through the rows of the merged repairs dataframe
for index, row in merged_repairs.iterrows():
    #checks if the next repair can be completed within the labor constraint
    if sum_labor_used + row["time_in_minutes"] < total_labor_minutes:
        #adds the repair's adjusted value to the total
        sum_repair_value += row['adjusted_value']
        #adds the repair's use of labor to the total
        sum_labor_used += row["time_in_minutes"]
    else:
        # when the next repair can't be fully completed within the labor constraint then the value from applying the remaining labor is calculated and added instead
        sum_repair_value += row['adjusted_value'] * ( (total_labor_minutes-sum_labor_used) / row['time_in_minutes'])
        break
#rounding to two decimal places
sum_repair_value = round(sum_repair_value, 2)
#printing calculated value
print(f'The maximized value from repairs over the next two weeks is: {sum_repair_value}')


The maximized value from repairs over the next two weeks is: 778282.93


Problem #2

In [2]:
#using pandas for data manipulation
import pandas as pd
#opening csv as dataframe
repair_types = pd.read_csv('repair_types.csv')

#adjusted value with severity = 1 would just be the repair value
#calculation for the repair value per minute of labor for each type of repair
repair_types['value_per_minute'] = repair_types['repair_value'] / repair_types['time_in_minutes']

#finding the index of the row where the value in the value per minute column is highest
highest_value_per_minute = repair_types.loc[repair_types['value_per_minute'].idxmax()]
#finding the value in the repair_id column with the index of the row we identified as having the highest value per minute value
best_repair_id = highest_value_per_minute['repair_id']

#printing the repair id string that we found in the same row as the highest value per minute
print(f'The most efficient repair to do is repair id: {best_repair_id}')

The most efficient repair to do is repair id: 8f7e2c3c-7390-4857-bda4-6c301e3c1e3b
